In [105]:
%matplotlib notebook

#every line of code in the first 4 boxes is from mirs covid_plot unless stated otherwise 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy.stats as st
from scipy.stats import linregress
import gmaps
from ipywidgets.embed import embed_minimal_html
import seaborn as sns
import plotly.express as px
import re
import gmaps.geojson_geometries
from matplotlib.cm import viridis
from matplotlib.cm import Spectral
from matplotlib.colors import to_hex
import country_converter as coco

#this will be will be the stuff ill be using on the animated graphs. im adding them as i see a tutorial
#Not necessarilly deteling them like those on top. In which I can guarantee i am not using half of those.
#I will only cite the ones that ends up working
import matplotlib as cm
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import pandas_alive
from celluloid import Camera
from matplotlib.animation import FuncAnimation
import plotly.express as px
import plotly.io as pio
from IPython.display import HTML
from tqdm import tqdm

import bar_chart_race as bcr

In [106]:
covid_stats_df =  pd.read_csv("covid_stats_2020.csv")
covid_stats_df.head()



,standard_names,ISO_2,ISO_3,Population (2020),lat,lng,Confirmed,stringency_index,Infection_rate_f
0,Afghanistan,AF,AFG,39074280,33.939110,67.709953,46274,42.700514,0.11
1,Albania,AL,ALB,2877239,41.153332,20.168331,52004,55.330056,1.81
2,Algeria,DZ,DZA,43984569,28.033886,1.659626,94371,59.981444,0.22
3,Andorra,AD,AND,77287,42.506285,1.521801,7519,40.286864,9.72
4,Angola,AO,AGO,33032075,-11.202692,17.873887,16562,57.541104,0.04


In [107]:
ultimate_no_null_df = pd.read_csv("ultimate_no_null.csv")
ultimate_no_null_df.head(-1)

#I need dates


,standard_names,month,year,Confirmed,stringency_index,lat,lng,Population (2020),Infection_rate,ISO_2,ISO_3
0,Afghanistan,1,2020,0,0.000000,33.939110,67.709953,39074280,0.00%,AF,AFG
1,Afghanistan,2,2020,1,2.010690,33.939110,67.709953,39074280,0.00%,AF,AFG
2,Afghanistan,3,2020,174,40.831935,33.939110,67.709953,39074280,0.00%,AF,AFG
3,Afghanistan,4,2020,1952,80.740000,33.939110,67.709953,39074280,0.01%,AF,AFG
4,Afghanistan,5,2020,13081,84.260000,33.939110,67.709953,39074280,0.03%,AF,AFG
...,...,...,...,...,...,...,...,...,...,...,...
2017,Zimbabwe,7,2020,2578,76.583548,-19.015438,29.154857,14899771,0.02%,ZW,ZWE
2018,Zimbabwe,8,2020,3328,80.560000,-19.015438,29.154857,14899771,0.02%,ZW,ZWE
2019,Zimbabwe,9,2020,1341,78.457667,-19.015438,29.154857,14899771,0.01%,ZW,ZWE
2020,Zimbabwe,10,2020,529,71.300000,-19.015438,29.154857,14899771,0.00%,ZW,ZWE


In [108]:
months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]

In [109]:
#I want to see the median and mean of all the column
#My stuff from here on out

coviddescr = covid_stats_df.describe()
coviddescr


,Population (2020),lat,lng,Confirmed,stringency_index,Infection_rate_f
count,1.700000e+02,170.000000,170.000000,1.700000e+02,170.000000,170.000000
mean,4.557895e+07,19.677676,20.417546,4.999643e+05,50.128176,1.475176
std,1.574453e+08,24.409918,60.812523,1.756754e+06,11.751698,1.924584
min,3.394400e+04,-40.900557,-106.346771,1.000000e+00,11.835054,0.000000
25%,4.049400e+06,4.933349,-5.159352,6.267750e+03,42.785932,0.092500
50%,1.056321e+07,18.853440,20.972652,5.282850e+04,51.459510,0.670000
75%,3.304568e+07,39.957297,47.553137,2.057332e+05,58.515895,2.352500
max,1.440298e+09,64.963051,178.065032,1.745930e+07,75.053683,10.160000


In [110]:
# popmode = covid_stats_df["Population (2020)"].mode
# stringmode = covid_stats_df["stringency_index"].mode
# irmode = covid_stats_df["Infection_rate_f"].mode

# Modedict = {"Population Mode": popmode,
#            "Stringency Mode": stringmode,
#            "Infection Rate Mode": irmode}
# print(Modedict)

#While there is a mode. But due to the amount of the mode. it may not be usuable. Commenting this line out.

In [111]:
# covid_stats_df.sort_values('stringency_index', )


In [112]:
#locating countries with a population less than 2 million

dropcrit = covid_stats_df[(covid_stats_df['Population (2020)'] < 2000000 )].index 
covid_stats_df.drop(dropcrit, inplace = True) 
covid_stats_df



#drop

,standard_names,ISO_2,ISO_3,Population (2020),lat,lng,Confirmed,stringency_index,Infection_rate_f
0,Afghanistan,AF,AFG,39074280,33.939110,67.709953,46274,42.700514,0.11
1,Albania,AL,ALB,2877239,41.153332,20.168331,52004,55.330056,1.81
2,Algeria,DZ,DZA,43984569,28.033886,1.659626,94371,59.981444,0.22
4,Angola,AO,AGO,33032075,-11.202692,17.873887,16562,57.541104,0.04
5,Argentina,AR,ARG,45267449,-38.416097,-63.616672,1531374,71.199127,3.40
...,...,...,...,...,...,...,...,...,...
165,Venezuela,VE,VEN,28421581,6.423750,-66.589730,109395,67.828683,0.38
166,Vietnam,VN,VNM,97490013,14.058324,108.277199,1410,56.785964,0.00
167,Yemen,YE,YEM,29935468,15.552727,48.516388,4023,29.630525,0.01
168,Zambia,ZM,ZMB,18468257,-13.133897,27.849332,18575,38.758279,0.10


In [113]:
#sorting the the dataframe descending
CovidDESC = covid_stats_df.sort_values(by=['stringency_index'], ascending = False).reset_index()
CovidDESC.head(10)



,index,standard_names,ISO_2,ISO_3,Population (2020),lat,lng,Confirmed,stringency_index,Infection_rate_f
0,17,Bolivia,BO,BOL,11700207,-16.290154,-63.588653,149149,75.053683,1.29
1,66,Honduras,HN,HND,9931333,15.199999,-86.241905,118312,73.698875,1.19
2,5,Argentina,AR,ARG,45267449,-38.416097,-63.616672,1531374,71.199127,3.40
3,88,Libya,LY,LBY,6887247,26.335100,17.228331,93772,70.094250,1.37
4,32,China,CN,CHN,1440297825,35.861660,104.195397,6665209,68.391841,0.46
5,165,Venezuela,VE,VEN,28421581,6.423750,-66.589730,109395,67.828683,0.38
6,49,Eritrea,ER,ERI,3554797,15.179384,39.782334,741,67.302524,0.00
7,119,Peru,PE,PER,33050211,-9.189967,-75.015152,989457,66.874263,3.01
8,120,Philippines,PH,PHL,109830324,12.879721,121.774017,456562,66.360124,0.42
9,10,Bangladesh,BD,BGD,164972348,23.684994,90.356331,498293,65.883020,0.30


In [114]:
#wanted to verify if those infection rate are 0% or something closer to zero. They are not 0. 

# dropcrit = covid_stats_df[(covid_stats_df['Population (2020)'] < 2000000 ) & (covid_stats_df['Infection_rate_f'] <= 0 )].index 
# covid_stats_df.drop(dropcrit, inplace = True) 
# covid_stats_df
# covid_stats_df.sort_values(by=['Infection_rate_f'], ascending = ['True'])


In [115]:
CovidASC = covid_stats_df.sort_values(by=['stringency_index'], ascending = True).reset_index()
CovidASC.head(10)

,index,standard_names,ISO_2,ISO_3,Population (2020),lat,lng,Confirmed,stringency_index,Infection_rate_f
0,24,Burundi,BI,BDI,11948454,-3.373056,29.918886,751,11.835054,0.00
1,109,Nicaragua,NI,NIC,6638075,12.865416,-85.207229,5938,12.433962,0.09
2,12,Belarus,BY,BLR,9448772,53.709807,27.953389,169648,13.849461,1.79
3,150,Tanzania,TZ,TZA,60012400,-6.369028,34.888822,509,21.555584,0.00
4,110,Niger,NE,NER,24346468,17.607789,8.081666,3696,23.502411,0.00
5,148,Taiwan,TW,TWN,23824369,23.697810,120.960515,1629,24.497490,0.00
6,167,Yemen,YE,YEM,29935468,15.552727,48.516388,4023,29.630525,0.01
7,137,Somalia,SO,SOM,15965848,5.152149,46.199616,4662,32.949035,0.02
8,149,Tajikistan,TJ,TJK,9573310,38.861034,71.276093,12888,33.469731,0.13
9,77,Japan,JP,JPN,126407422,36.204824,138.252924,193753,33.586064,0.15


In [116]:
#I made the sort according to stringency index

In [117]:
CovidASCIR = covid_stats_df.sort_values(by=['Infection_rate_f'], ascending = True).reset_index()
CovidASCIR.head(10)

,index,standard_names,ISO_2,ISO_3,Population (2020),lat,lng,Confirmed,stringency_index,Infection_rate_f
0,41,DR Congo,CD,COD,90003954,-4.038333,21.758664,15089,44.189288,0.0
1,49,Eritrea,ER,ERI,3554797,15.179384,39.782334,741,67.302524,0.0
2,166,Vietnam,VN,VNM,97490013,14.058324,108.277199,1410,56.785964,0.0
3,83,Laos,LA,LAO,7293542,19.856270,102.495496,41,34.867858,0.0
4,30,Chad,TD,TCD,16502877,15.454166,18.732207,1839,54.709095,0.0
5,151,Thailand,TH,THA,69830779,15.870032,100.992541,4331,45.480477,0.0
6,150,Tanzania,TZ,TZA,60012400,-6.369028,34.888822,509,21.555584,0.0
7,148,Taiwan,TW,TWN,23824369,23.697810,120.960515,1629,24.497490,0.0
8,110,Niger,NE,NER,24346468,17.607789,8.081666,3696,23.502411,0.0
9,26,Cambodia,KH,KHM,16758448,12.565679,104.990963,362,37.321304,0.0


In [118]:
CovidDESCIR = covid_stats_df.sort_values(by=['Infection_rate_f'], ascending = False).reset_index()
CovidDESCIR.head(10)

,index,standard_names,ISO_2,ISO_3,Population (2020),lat,lng,Confirmed,stringency_index,Infection_rate_f
0,55,France,FR,FRA,65298930,46.227638,2.213749,5352449,55.223795,8.21
1,40,Czech Republic,CZ,CZE,10712481,49.817492,15.472962,610006,44.911555,5.70
2,13,Belgium,BE,BEL,11598451,50.503887,4.469936,621039,51.157601,5.37
3,161,United States,US,USA,331341050,37.090240,-95.712891,17459296,56.045880,5.27
4,58,Georgia,GE,GEO,3987805,32.165622,-82.900075,204003,59.679837,5.12
5,135,Slovenia,SI,SVN,2078989,46.151241,14.995463,103555,47.300186,5.00
6,123,Qatar,QA,QAT,2889284,25.354826,51.183884,141716,60.872639,4.93
7,141,Spain,ES,ESP,46757980,40.463667,-3.749220,2246032,55.975557,4.81
8,116,Panama,PA,PAN,4326296,8.537981,-80.782127,206310,63.958736,4.76
9,146,Switzerland,CH,CHE,8665615,46.818188,8.227512,403989,41.892448,4.66


In [119]:
#ultimate_no_null_df
#The df with countries

In [120]:
ConfirmedDESC = ultimate_no_null_df.sort_values(by=['Confirmed'], ascending = False).reset_index()
ConfirmedDESC.head(10)
#sorting the other data fram. Puprposely not removing the population on this one. Will  do so later if this works
#Looks like India, and the US is the most in this.
#This is really irrelavant because its not group, but it also tells you the month.


,index,standard_names,month,year,Confirmed,stringency_index,lat,lng,Population (2020),Infection_rate,ISO_2,ISO_3
0,389,China,9,2020,6486858,60.092667,35.861660,104.195397,1440297825,0.45%,CN,CHN
1,1926,United States,11,2020,4462302,69.628667,37.090240,-95.712891,331341050,1.35%,US,USA
2,1927,United States,12,2020,3859800,72.044615,37.090240,-95.712891,331341050,1.16%,US,USA
3,819,India,9,2020,2621418,79.105000,20.593684,78.962880,1382345085,0.19%,IN,IND
4,653,France,11,2020,2501745,78.330000,46.227638,2.213749,65298930,3.83%,FR,FRA
5,818,India,8,2020,1995178,79.764516,20.593684,78.962880,1382345085,0.14%,IN,IND
6,1922,United States,7,2020,1923898,68.263871,37.090240,-95.712891,331341050,0.58%,US,USA
7,1925,United States,10,2020,1917201,64.051613,37.090240,-95.712891,331341050,0.58%,US,USA
8,820,India,10,2020,1871498,68.083548,20.593684,78.962880,1382345085,0.14%,IN,IND
9,1923,United States,8,2020,1463760,67.130000,37.090240,-95.712891,331341050,0.44%,US,USA


In [121]:
ConfirmedDESC = ultimate_no_null_df.sort_values(by=['Confirmed'], ascending = True).reset_index()
ConfirmedDESC.head(10)

#sorting the other data fram. Puprposely not removing the population on this one. Will  do so later if this works
#This should be obvious since this is the beginning. This is not really relevant. Unless its group by countries

,index,standard_names,month,year,Confirmed,stringency_index,lat,lng,Population (2020),Infection_rate,ISO_2,ISO_3
0,0,Afghanistan,1,2020,0,0.000000,33.939110,67.709953,39074280,0.00%,AF,AFG
1,1212,Morocco,1,2020,0,0.000000,31.791702,-7.092620,36985624,0.00%,MA,MAR
2,595,Estonia,1,2020,0,0.000000,58.595272,25.013607,1326693,0.00%,EE,EST
3,587,Eritrea,5,2020,0,93.520000,15.179384,39.782334,3554797,0.00%,ER,ERI
4,584,Eritrea,2,2020,0,0.000000,15.179384,39.782334,3554797,0.00%,ER,ERI
5,583,Eritrea,1,2020,0,0.000000,15.179384,39.782334,3554797,0.00%,ER,ERI
6,1666,Spain,1,2020,0,0.358387,40.463667,-3.749220,46757980,0.00%,ES,ESP
7,572,El Salvador,2,2020,0,19.440000,13.794185,-88.896530,6491923,0.00%,SV,SLV
8,571,El Salvador,1,2020,0,3.494194,13.794185,-88.896530,6491923,0.00%,SV,SLV
9,1679,Sri Lanka,2,2020,0,8.905172,7.873054,80.771797,21428970,0.00%,LK,LKA


In [122]:
#Race Bar Chart

Pivot = ultimate_no_null_df.pivot(index='month', columns='standard_names', values='Confirmed').reset_index()

Pivot

#source = https://chrisalbon.com/python/data_wrangling/pandas_long_to_wide/


#changing the data to wide.
# With month as the Index, the countries as the columns, and the Confirmed Cases as the values

standard_names,month,Afghanistan,Albania,Algeria,Andorra,Angola,Argentina,Australia,Austria,Azerbaijan,...,United Kingdom,United States,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,Yemen,Zambia,Zimbabwe
0,1,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,...,2.0,8.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
1,2,1.0,0.0,1.0,0.0,0.0,0.0,16.0,9.0,0.0,...,199.0,17.0,0.0,0.0,0.0,0.0,14.0,0.0,0.0,0.0
2,3,174.0,243.0,715.0,376.0,7.0,1054.0,4534.0,10171.0,298.0,...,132701.0,192152.0,338.0,172.0,0.0,135.0,196.0,0.0,35.0,8.0
3,4,1952.0,530.0,3290.0,369.0,20.0,3374.0,2207.0,5272.0,1506.0,...,236937.0,884047.0,806.0,1867.0,0.0,198.0,58.0,6.0,71.0,32.0
4,5,13081.0,364.0,5388.0,19.0,59.0,12423.0,436.0,1279.0,3690.0,...,78768.0,718241.0,180.0,1584.0,0.0,1177.0,58.0,317.0,951.0,178.0
5,6,16299.0,1398.0,4513.0,91.0,198.0,47679.0,718.0,1035.0,12030.0,...,27677.0,834974.0,113.0,4880.0,0.0,4322.0,27.0,835.0,537.0,413.0
6,7,5158.0,2741.0,16487.0,70.0,864.0,126772.0,9360.0,3364.0,14354.0,...,19577.0,1923898.0,328.0,15506.0,0.0,12742.0,203.0,570.0,4369.0,2578.0
7,8,1494.0,4237.0,14100.0,251.0,1506.0,226433.0,8539.0,6308.0,4557.0,...,33290.0,1463760.0,331.0,17884.0,0.0,28154.0,486.0,2062.0,6134.0,3328.0
8,9,1109.0,4136.0,7036.0,874.0,2318.0,333266.0,1277.0,17375.0,3794.0,...,117763.0,1202896.0,451.0,14824.0,0.0,28394.0,50.0,76.0,2662.0,1341.0
9,10,2157.0,7226.0,6412.0,2706.0,5833.0,415923.0,499.0,60112.0,15040.0,...,558947.0,1917201.0,1078.0,10215.0,0.0,16891.0,86.0,29.0,1673.0,529.0


In [123]:
#source = https://www.youtube.com/watch?v=eF9VH4zKoTI
#source2 = https://github.com/dexplo/

race = bcr.bar_chart_race(Pivot, 
                   n_bars = 10,steps_per_period=10,period_length=3000,  
                   title='COVID-19 Monthly Confirmed Cases by Country',     
                   period_summary_func=lambda v, r: {'x': .98, 'y': .2, 's': f'Total Cases: {v.sum():,.0f}', 'ha': 'right', 'size': 11}
#                    ,filename='../ovid19racing.gif'
                  )

race

#wait about 1 to 2 minutes

C:\Users\danza\.conda\envs\PythonData\lib\site-packages\matplotlib\backends\backend_agg.py:238: RuntimeWarning: Glyph 127 missing from current font.
  font.set_text(s, 0.0, flags=flags)
C:\Users\danza\.conda\envs\PythonData\lib\site-packages\matplotlib\backends\backend_agg.py:238: RuntimeWarning: Glyph 128 missing from current font.
  font.set_text(s, 0.0, flags=flags)
C:\Users\danza\.conda\envs\PythonData\lib\site-packages\matplotlib\backends\backend_agg.py:238: RuntimeWarning: Glyph 129 missing from current font.
  font.set_text(s, 0.0, flags=flags)
C:\Users\danza\.conda\envs\PythonData\lib\site-packages\matplotlib\backends\backend_agg.py:238: RuntimeWarning: Glyph 130 missing from current font.
  font.set_text(s, 0.0, flags=flags)
C:\Users\danza\.conda\envs\PythonData\lib\site-packages\matplotlib\backends\backend_agg.py:238: RuntimeWarning: Glyph 131 missing from current font.
  font.set_text(s, 0.0, flags=flags)
C:\Users\danza\.conda\envs\PythonData\lib\site-packages\matplotlib\bac

C:\Users\danza\.conda\envs\PythonData\lib\site-packages\bar_chart_race\_make_chart.py:226: UserWarning: Some of your columns never make an appearance in the animation. To reduce color repetition, set `filter_column_colors` to `True`
  warnings.warn("Some of your columns never make an appearance in the animation. "


In [102]:
# 

AttributeError: 'HTML' object has no attribute 'savefig'